Nomes: Bel Cogo e Bruno da Siqueira Hoffmann.

Professor: Felipe de Morais.

Disciplina: Ciência de Dados e Big Data.

# Tarefa 6

Para a tarefa em questão foi utilizado o banco NoSQL MongoDB, além disso, foi usado a base de dados: https://www.kaggle.com/datasets/evgeny1928/playstation-games-info?select=games.csv. Usando o modelo `.csv` e executando os scripts abaixo para realizar a conversão do `.csv` em `.json`. A partir disso foi feita as configurações expostas na aula para execução do banco de dados Mongo e a inserção da collection.

In [1]:
import csv
import json

def csv_to_json(csv_file_path, json_file_path):
    # Abrir o arquivo CSV e ler o conteúdo
    with open(csv_file_path, mode='r', encoding='utf-8') as csv_file:
        csv_reader = csv.DictReader(csv_file)
        data = []
        
        # Converter cada linha do CSV em um dicionário e adicionar à lista
        for row in csv_reader:
            # Converter strings de listas para listas reais
            if 'platforms' in row:
                row['platforms'] = eval(row['platforms'])
            if 'genres' in row:
                row['genres'] = eval(row['genres'])
            if 'price' in row:
                row['price'] = float(row['price'])
            if 'rating' in row:
                row['rating'] = float(row['rating'])
                
            # Adicionar o dicionário à lista
            data.append(row)
    
    # Salvar a lista de dicionários no arquivo JSON
    with open(json_file_path, mode='w', encoding='utf-8') as json_file:
        json.dump(data, json_file, indent=4, ensure_ascii=False)

# Exemplo de uso:
csv_to_json('games.csv', 'games.json')


Após transformar o arquivo `.csv` em um arquivo `.json` foi necessário executar os comandos para importar o arquivo `json` para o banco de dados como uma collection, esses comando sendo executados em um terminal.

In [3]:
mongoimport --db playstation --collection games --file games.json --jsonArray

SyntaxError: invalid syntax (1657558742.py, line 1)

Em seguida, foi executado o comando python abaixo para se conectar com o banco de dados.

In [6]:
import json
from pymongo import MongoClient
import pandas as pd


# se conecta com o servidor mongo da maquina
client = MongoClient('localhost', 27017)
   
# lista todos os databases
client.list_database_names()

# le a coleção denominada playstation
db = client.playstation

# desta coleção, le os arquivos de games
movies = db.games

## Pergunta 1

**Pergunta:** Quais são os jogos com os generos ação ou aventura e com a nota maior que 4.5? Indique o nome, a nota, o preço e os demais generos.

**Resposta:**

In [9]:
criteria = {
  '$or': [
    {'genres': { '$in': ['Action'] }},
    {'genres': { '$in': ['Adventure'] }}
  ],
  'rating': { '$gt': 4.5 }
}

projection = {
    'title': 1,
    'rating': 1,
    'genres': 1,
    'price': 1,
    '_id': 0
}


cursor = movies.find(criteria, projection)
df = pd.DataFrame(cursor)

df

,title,rating,price,genres
0,Grand Theft Auto Online (PlayStation®5),4.52,19.99,"[Action, Adventure]"
1,Marvel’s Spider-Man 2,4.86,49.69,[Action]
2,Honkai: Star Rail,4.60,0.00,"[Adventure, Role Playing Games]"
3,Red Dead Redemption 2,4.71,19.79,"[Action, Adventure, Unique]"
4,Stellar Blade™,4.82,69.99,"[Adventure, Action]"
...,...,...,...,...
1017,The Demon Rush: Legends Corrupt,5.00,17.99,"[Action, Role Playing Games]"
1018,The Journey Down: Chapter One,4.81,6.99,"[Adventure, Adventure]"
1019,The Last Door - Complete Edition,4.66,14.99,"[Adventure, Horror, Puzzle]"
1020,The Legend of Tianding,4.59,19.99,"[Adventure, Action]"


## Pergunta 2

**Pergunta:** Qual é o nome e o preço do jogo mais bem avaliado?

**Resposta:**

## Pergunta 3

**Pergunta:** Qual gênero de jogo é o mais caro? 

**Resposta:**

## Pergunta 4

**Pergunta:** Qual o mês com mais lançamentos de jogos?

**Resposta:**

## Pergunta 5

**Pergunta:** Qual plataforma é a mais popular?

**Resposta:**